In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ModuleNotFoundError as e:
    print("not in colab")
    pass
import os
base_dir = "/content/drive/MyDrive/semeval2022"
if not os.path.exists(base_dir):
  !pip install -r requirements.txt
  base_dir = ""
else:
  !pip install -r /content/drive/MyDrive/semeval2022/requirements.txt
  !cp -rf /content/drive/MyDrive/semeval2022/*.py . 
  !cp -rf /content/drive/MyDrive/semeval2022/utils .
  !cp -rf /content/drive/MyDrive/semeval2022/model .

In [ ]:
from utils.util import get_reader, train_model, save_model, parse_args, get_tagset, load_model, wnut_iob
import time

In [ ]:
encoder_model = "roberta-base"
checkpoint = os.path.join(base_dir, "roberta-train/lightning_logs/version_0/checkpoints/roberta-train_timestamp_1638429490.282762_final.ckpt")
train_file = os.path.join(base_dir, "training_data/EN-English/en_train.conll")
dev_file = os.path.join(base_dir, "training_data/EN-English/en_dev.conll")
output_dir = os.path.join(base_dir, "roberta-finetune")
iob_tagging = wnut_iob

In [ ]:
train_data = get_reader(file_path=train_file, target_vocab=wnut_iob, encoder_model=encoder_model, max_instances=-1, max_length=50)
dev_data = get_reader(file_path=dev_file, target_vocab=wnut_iob, encoder_model=encoder_model, max_instances=-1, max_length=50)
model = load_model(checkpoint, tag_to_id=wnut_iob, stage='finetune')
model.train_data = train_data
model.dev_data = dev_data
trainer = train_model(model=model, out_dir=output_dir, epochs=5, monitor='f1')
out_model_path = save_model(trainer=trainer, out_dir=output_dir, model_name="finetune", timestamp=time.time())